In [1]:
import pandas as pd
import numpy as np
import os
import imageio
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,Flatten,Activation,Dropout,Maximum,ZeroPadding2D,Conv2D,MaxPooling2D,concatenate,Add,BatchNormalization,LeakyReLU
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize as imresize
from tqdm import tqdm
import matplotlib.pyplot as plt


from subprocess import check_output

In [2]:
# Dense layers set
def dense_set(inp_layer, n, activation, drop_rate=0.):
    dp = Dropout(drop_rate)(inp_layer)
    dns = Dense(n)(dp)
    bn = BatchNormalization(axis=-1)(dns)
    act = Activation(activation=activation)(bn)
    return act

# Conv layers set
def conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False):
    if zp_flag:
        zp = ZeroPadding2D((1,1))(feature_batch)
    else:
        zp = feature_batch
    conv = Conv2D(filters=feature_map, kernel_size=kernel_size, strides=strides,padding='same')(zp)
    bn = BatchNormalization(axis=3)(conv)
    act = LeakyReLU(1/10)(bn)
    return act
# residual_block
def residual_block(feature_batch, feature_map,stride=(1,1)):
    res=conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=stride, zp_flag=False)
    res=conv_layer(res, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False)
    shortcut=Conv2D(filters=feature_map, kernel_size=(3,3), strides=stride,padding='same')(feature_batch)
    shortcut = BatchNormalization()(shortcut)
    output=Add()([shortcut,res])
    output=LeakyReLU(1/10)(output)
    return output

In [3]:
# simple model 
def get_model():
    inp_img = Input(shape=(128, 64, 3))

    # 32
    conv1 = residual_block(inp_img, 32)
    conv2 = residual_block(conv1, 32)
    # 64
    conv3 = residual_block(conv2, 64,stride=(2,2))
    conv4 = residual_block(conv3, 64)
    # 128
    conv5 = residual_block(conv4, 128,stride=(2,2))
    conv6 = residual_block(conv5, 128)
    conv7 = residual_block(conv6, 128)
    mp3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),padding='same')(conv7)

    flt=tf.keras.layers.GlobalAveragePooling2D()(mp3)
    ds1 = dense_set(flt, 128, activation='tanh')
    feature=tf.math.l2_normalize(ds1, axis=1)
    out = dense_set(ds1, 1500, activation='softmax')

    model = Model(inputs=inp_img, outputs=[out,feature])
    
    # The first 50 epochs are used by Adam opt.
    # Then 30 epochs are used by SGD opt.
    
    #mypotim = Adam(lr=2 * 1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    mypotim = SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                   optimizer=mypotim,
                   metrics=['accuracy'])
    model.summary()
    return model

In [4]:
def test_model(img, label):
    gmodel = get_model()
    gmodel.load_weights(filepath='model/model.47-1.00.hdf5')
    prob = gmodel.predict(img, verbose=1)
    pred = prob[0].argmax(axis=-1)
    #sub = pd.DataFrame({"file": label,
    #                     "species": [INV_CLASS[p] for p in pred]})
    #sub.to_csv("sub.csv", index=False, header=True)
    pred=np.array(pred)

    count=0
    for i in range(len(pred)):
        if pred[i]==label[i]:
            count+=1
    print(float(count/len(pred)))
    logits=prob[1]
    print(logits.shape)

In [5]:
import re
# Resize all image to 128x64
def img_reshape(img):
    img = imresize(img, (128, 64, 3))
    return img

# get image tag
def img_label(path):
    path=path.split('/')[-1]
    return int(re.split(r'\\',path)[-2])

# fill train and test dict
def fill_dict(paths, some_dict):
    text = ''
    text = 'Start fill test_dict'
    
    for p in tqdm(paths, ascii=True, ncols=85, desc=text):
        img = imageio.imread(p)
        img = img_reshape(img)
        some_dict['image'].append(img)
        some_dict['label'].append(img_label(p))

    return some_dict

In [6]:
# read image from dir. and fill test dict
def reader():
    file_ext = []
    train_path = []
    test_path = []

    for root, dirs, files in os.walk('dataset_seperate/test/'):
        if dirs != []:
            print('Root:\n'+str(root))
            print('Dirs:\n'+str(dirs))
        else:
            for f in files:
                ext = os.path.splitext(str(f))[1][1:]
                if ext not in file_ext:
                    file_ext.append(ext)
                path = os.path.join(root, f)
                test_path.append(path)
    test_dict = {
        'image': [],
        'label': []
    }
    test_dict = fill_dict(test_path, test_dict)
    return  test_dict

In [7]:
# I commented out some of the code for learning the model.
def main():
    test_dict = reader()
    X_test = np.array(test_dict['image'])
    label = np.array(test_dict['label'])
    # I do not recommend trying to train the model on a kaggle.
    #train_model(X_train, y_train)
    test_model(X_test, label)

if __name__=='__main__':
    main()

39;1406&#39;, &#39;1407&#39;, &#39;1408&#39;, &#39;1409&#39;, &#39;1410&#39;, &#39;1411&#39;, &#39;1412&#39;, &#39;1413&#39;, &#39;1414&#39;, &#39;1415&#39;, &#39;1416&#39;, &#39;1417&#39;, &#39;1418&#39;, &#39;1419&#39;, &#39;1420&#39;, &#39;1421&#39;, &#39;1422&#39;, &#39;1423&#39;, &#39;1424&#39;, &#39;1425&#39;, &#39;1426&#39;, &#39;1427&#39;, &#39;1428&#39;, &#39;1429&#39;, &#39;1430&#39;, &#39;1431&#39;, &#39;1432&#39;, &#39;1433&#39;, &#39;1434&#39;, &#39;1435&#39;, &#39;1436&#39;, &#39;1437&#39;, &#39;1438&#39;, &#39;1439&#39;, &#39;1440&#39;, &#39;1441&#39;, &#39;1442&#39;, &#39;1443&#39;, &#39;1444&#39;, &#39;1445&#39;, &#39;1446&#39;, &#39;1447&#39;, &#39;1448&#39;, &#39;1449&#39;, &#39;1450&#39;, &#39;1451&#39;, &#39;1452&#39;, &#39;1453&#39;, &#39;1454&#39;, &#39;1455&#39;, &#39;1456&#39;, &#39;1457&#39;, &#39;1458&#39;, &#39;1459&#39;, &#39;1460&#39;, &#39;1461&#39;, &#39;1462&#39;, &#39;1463&#39;, &#39;1464&#39;, &#39;1465&#39;, &#39;1466&#39;, &#39;1467&#39;, &#39;1468&